In [ ]:
# WARNING: advised to install a specific version, e.g. tensorwaves==0.1.2
%pip install -q tensorwaves[doc,jax,pwa,viz] IPython

In [ ]:
%config InlineBackend.figure_formats = ['svg']
import os

from IPython.display import display  # noqa: F401

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# Speed up lambdifying

In [ ]:
import logging

import ampform
import graphviz
import qrules
import sympy as sp
from ampform.dynamics.builder import create_relativistic_breit_wigner_with_ff
from IPython.display import HTML, SVG

from tensorwaves.function.sympy import (
    create_parametrized_function,
    fast_lambdify,
    split_expression,
)

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)

:::{note}

Since [#374](https://github.com/ComPWA/tensorwaves/pull/374), expressions are lambdified with common sub-expressions. This should already reduce lambdification time significantly and also results in faster computational functions.

:::

## Split expression

Lambdifying a SymPy expression can take rather long when an expression is complicated. Fortunately, TensorWaves offers a way to speed up the lambdify process. The idea is to split up an an expression into sub-expressions, separate those separately, and then recombining them. Let's illustrate that idea with the following simplified example:

In [ ]:
x, y, z = sp.symbols("x:z")
expr = x**z + 2 * y + sp.log(y * z)
expr

This expression can be represented in a tree of mathematical operations.

In [ ]:
dot = sp.dotprint(expr, bgcolor="none")
graphviz.Source(dot)

The function {func}`.split_expression` can now be used to split up this expression tree into a 'top expression' plus definitions for each of the sub-expressions into which it was split:

In [ ]:
top_expr, sub_expressions = split_expression(expr, max_complexity=3)
top_expr

In [ ]:
sub_expressions

The original expression can easily be reconstructed with {meth}`~sympy.core.basic.Basic.subs` or {meth}`~sympy.core.basic.Basic.xreplace`:

In [ ]:
top_expr.xreplace(sub_expressions)

Each of the expression trees are now smaller than the original:

In [ ]:
dot = sp.dotprint(top_expr, bgcolor="none")
graphviz.Source(dot)

In [ ]:
for symbol, definition in sub_expressions.items():
    dot = sp.dotprint(definition, bgcolor="none")
    graph = graphviz.Source(dot)
    graph.render(filename=f"sub_expr_{symbol.name}", format="svg")

html = "<table>\n"
html += "  <tr>\n"
html += "".join(
    '    <th style="text-align:center;'
    f' background-color:white">{symbol.name}</th>\n'
    for symbol in sub_expressions
)
html += "  </tr>\n"
html += "  <tr>\n"
for symbol in sub_expressions:
    svg = SVG(f"sub_expr_{symbol.name}.svg").data
    html += f'    <td style="background-color:white">{svg}</td>\n'
html += "  </tr>\n"
html += "</table>"
HTML(html)

## Fast lambdify

Generally, the lambdify time scales exponentially with the size of an expression tree. With larger expression trees, it's therefore much faster to lambdify these sub-expressions separately and to recombine them. TensorWaves offers a function that does this for you: {func}`.fast_lambdify`. We'll use an {class}`~ampform.helicity.HelicityModel` to illustrate this:

In [ ]:
reaction = qrules.generate_transitions(
    initial_state=("J/psi(1S)", [+1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
)
model_builder = ampform.get_builder(reaction)
for name in reaction.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.formulate()

In [ ]:
expression = model.expression.doit()
sorted_symbols = sorted(expression.free_symbols, key=lambda s: s.name)

```{autolink-skip}
```

In [ ]:
%%time
split_function = fast_lambdify(
    expression,
    sorted_symbols,
    max_complexity=100,
    backend="numpy",
)

In [ ]:
split_function

```{autolink-skip}
```

In [ ]:
%%time
normal_function = sp.lambdify(sorted_symbols, expression)

In [ ]:
normal_function

## Specifying complexity

When creating a parametrized function, we use the {func}`.create_parametrized_function` function. By default, this internally calls SymPy's own {func}`~sympy.utilities.lambdify.lambdify` function. But if you specify its `max_complexity` argument, {func}`.create_parametrized_function` uses TensorWaves's {func}`.fast_lambdify`.

```{autolink-skip}
```

In [ ]:
%%time
function = create_parametrized_function(
    expression=model.expression.doit(),
    parameters=model.parameter_defaults,
    max_complexity=100,
    backend="numpy",
)